In [13]:
from pycocotools.coco import COCO

import torch
import torch.nn as nn

import torchvision.models as models

import numpy as np

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
resnet = models.resnet101(pretrained=True)
resnet

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to C:\Users\Shayan/.cache\torch\hub\checkpoints\resnet101-5d3b4d8f.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
list(resnet.children())[-1]

Linear(in_features=2048, out_features=1000, bias=True)

In [17]:
class Encoder(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(Encoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        resnet = models.resnet101(pretrained=True)
        
        for params in resnet.parameters(): # will not be fine-tuning resnet
            params.requires_grad = False
        
        self.resnet = nn.Sequential(*list(resnet.children()[:-1])) # will not be using last layer of resnet since that layer outputs a 1000-D vector for imagenet classification
        self.embedding = nn.Linear(2048, hidden_size) # add a linear layer to get a feature vector to pass into the decoder
        
    def forward(self, inputs):
        output = self.resnet(inputs)
        output = self.embedding(outputs)
        return output

In [18]:
class Decoder(nn.Module):
    
    def __init__(self, vocab_size, hidden_size):
        super(Decoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_size)
        self.rnn = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True) # word embeddings will be size hidden_size, hidden states will be size hidden_size, and batch_first=True since we want input dimensions (batch, seq, hidden)
        self.out = nn.Linear(in_features=hidden_size, out_features=vocab_size)
    
    def forward(self, inputs, encoder_out):
        """
        inputs: (batch, seq_len) set of captions, each entry is an integer index into the vocabulary
        encoder_out: (hidden_size,) vector of CNN encoder output
        """
        output = self.embedding(inputs)
        output = self.rnn(output, (encoder_out, encoder_out))
        output = self.out(output)
        return output